# Validation

In order to validate our results, this notebook compare our results with the flood mapping from the ['Global Flood Monitoring'(GFM)](https://services.eodc.eu/browser/#/v1/collections/GFM) for the same area and time range.

In [ ]:
from importlib.resources import files

import holoviews as hv
import hvplot.xarray  # noqa
import numpy as np
import pystac_client
import xarray as xr
from dask.distributed import Client
from dask_flood_mapper import flood
from odc import stac as odc_stac

In [ ]:
# define time range and area
time_range = "2023-10-11/2023-10-25"
bounding_box = [12.3, 54.3, 13.1, 54.6]

In [ ]:
client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="12GB")

## GFM

In [ ]:
# Connect to STAC catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1/")

# Search for available images

search = eodc_catalog.search(collections="GFM", bbox=bounding_box, datetime=time_range)
items_GFM = search.item_collection()

print(f"Found {len(items_GFM)} items")

### Load the data

In [ ]:
crs = "EPSG:4326"  # Coordinate Reference System - World Geodetic System 1984 (WGS84)
resolution = 0.00018  # 20 meter in degree

GFM_fd = odc_stac.load(
    items_GFM,
    bbox=bounding_box,
    crs=crs,
    bands=["tuw_likelihood", "tuw_flood_extent"],
    resolution=resolution,
    dtype="uint8",
    chunks={"x": 1000, "y": 1000, "time": -1},
)

### Pre-processing 

In [ ]:
# convert values 255 to NANs
GFM_fd["tuw_flood_extent"] = GFM_fd.tuw_flood_extent.where(
    GFM_fd.tuw_flood_extent != 255
).compute()

# select the band 'tuw_flood_extent'
GFM_fd = GFM_fd["tuw_flood_extent"]
GFM_fd

## dask-flood-mapper

In [ ]:
# calculate flood decision
fd = flood.decision(bbox=bounding_box, datetime=time_range).compute()
fd

### Preprocessing

Make both plots compatible to be plot together.

In [ ]:
data_text = files("dask_flood_mapper.data").joinpath("wcover.tif")
wcover = xr.open_dataarray(data_text, band_as_variable=True).rename(
    {"y": "latitude", "x": "longitude"}
)
wcover

In [ ]:
fd = fd.where(wcover != 80)
fd

## Validation plot

In [ ]:
common_times = np.intersect1d(GFM_fd.time.values, fd.time.values)


def synced_plot(t):
    plot1 = GFM_fd.sel(time=t).hvplot.image(
        x="longitude",
        y="latitude",
        title="GFM flood map",
        cmap=["rgba(0, 0, 1, 0.1)", "darkblue"],
    )
    plot2 = fd.sel(time=t).hvplot.image(
        x="longitude",
        y="latitude",
        title="dask-flood-mapper",
        cmap=["rgba(0, 0, 1, 0.1)", "darkblue"],
    )

    return (plot1 + plot2).cols(2)


combined_plot = hv.DynamicMap(synced_plot, kdims="time").redim.values(time=common_times)

combined_plot